
# importing libraries & dataset

In [ ]:
!pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd

from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import category_encoders as ce

In [ ]:
df = pd.read_csv("/kaggle/input/eal-estate-goldmine/dubai_properties.csv")


# printing information about df

In [ ]:
print(df.info())

In [ ]:
print(df.isna().sum())

In [ ]:
print(df.duplicated().sum())

# columns relationships

In [ ]:
# Property Type Distribution
plt.figure(figsize=(10,6))
ax = sns.countplot(x='Type', data=df, palette='muted')

for p in ax.patches:
    ax.annotate(f"{int(p.get_height()):,}", (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Property Type Distribution')
plt.xlabel('Property Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Relationship between Rent and Area in Square Feet
plt.figure(figsize=(10,6))
sns.scatterplot(x='Area_in_sqft', y='Rent', data=df, alpha=0.5)
plt.title('Rent vs. Area in Square Feet')
plt.xlabel('Area in Square Feet')
plt.ylabel('Rent (AED)')
plt.show()

In [ ]:
# Distribution of Rental Prices per Square Foot
plt.figure(figsize=(10,6))
sns.histplot(df['Rent_per_sqft'], bins=50, kde=True, color='salmon')
plt.title('Distribution of Rental Prices per Square Foot')
plt.xlabel('Rent per Square Foot (AED)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Rental Price Category Distribution by City
plt.figure(figsize=(12,6))
sns.countplot(x='City', hue='Rent_category', data=df, palette='husl')
plt.title('Rental Price Category Distribution by City')
plt.xlabel('City')
plt.ylabel('Count')
plt.legend(title='Rent Category')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Furnishing Status Distribution
plt.figure(figsize=(8,6))
ax = sns.countplot(x='Furnishing', data=df, palette='pastel')
plt.title('Furnishing Status Distribution')
plt.xlabel('Furnishing Status')
plt.ylabel('Count')
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                xytext = (0, 10),
                textcoords = 'offset points')

plt.show()

In [ ]:
# Age of Listing Distribution
plt.figure(figsize=(10,6))
sns.histplot(df['Age_of_listing_in_days'], bins=50, kde=True, color='lightgreen')
plt.title('Age of Listing Distribution')
plt.xlabel('Age of Listing (Days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Geographical Distribution of Rental Properties
plt.figure(figsize=(10,8))
sns.scatterplot(x='Longitude', y='Latitude', hue='City', data=df, palette='muted', alpha=0.5)
plt.title('Geographical Distribution of Rental Properties')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='City')
plt.show()

In [ ]:
# Age of Listing Distribution by City
plt.figure(figsize=(12,6))
sns.boxplot(x='City', y='Age_of_listing_in_days', data=df, palette='viridis')
plt.title('Age of Listing Distribution by City')
plt.xlabel('City')
plt.ylabel('Age of Listing (Days)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Average Rental Price by Number of Bedrooms
plt.figure(figsize=(10,6))
sns.barplot(x='Beds', y='Rent', data=df, palette='muted')
plt.title('Average Rental Price by Number of Bedrooms')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Average Rent (AED)')
plt.show()


In [ ]:
# Rental Price Category Distribution by Number of Bedrooms
plt.figure(figsize=(10,6))
sns.countplot(x='Beds', hue='Rent_category', data=df, palette='Set2')
plt.title('Rental Price Category Distribution by Number of Bedrooms')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Count')
plt.show()

In [ ]:
# Analyzing the relationship between rent and number of bedrooms
plt.figure(figsize=(10, 6))
sns.boxplot(x='Beds', y='Rent', data=df)
plt.title('Rent Prices by Number of Bedrooms')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Rent (AED)')
plt.yscale('log')
plt.show()

# Analyzing the relationship between rent and number of bathrooms
plt.figure(figsize=(10, 6))
sns.boxplot(x='Baths', y='Rent', data=df)
plt.title('Rent Prices by Number of Bathrooms')
plt.xlabel('Number of Bathrooms')
plt.ylabel('Rent (AED)')
plt.yscale('log')
plt.show()

# Analyzing the relationship between rent and area in sqft
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Area_in_sqft', y='Rent', data=df)
plt.title('Rent Prices by Area in Sqft')
plt.xlabel('Area in Sqft')
plt.ylabel('Rent (AED)')
plt.xscale('log')
plt.yscale('log')
plt.show()

# Analyzing the relationship between rent and property type
plt.figure(figsize=(12, 6))
sns.boxplot(x='Type', y='Rent', data=df)
plt.title('Rent Prices by Property Type')
plt.xlabel('Property Type')
plt.ylabel('Rent (AED)')
plt.yscale('log')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Analyzing how rent varies across different cities
plt.figure(figsize=(12, 8))
sns.boxplot(x='City', y='Rent', data=df)
plt.title('Rent Prices by City')
plt.xlabel('City')
plt.ylabel('Rent (AED)')
plt.yscale('log')
plt.xticks(rotation=45)
plt.show()

# Analyzing how rent varies across different locations within cities
# Due to potentially large number of locations, we display only the top 20 locations by median rent
top_locations = df.groupby('Location')['Rent'].median().sort_values(ascending=False).head(20).index
filtered_data = df[df['Location'].isin(top_locations)]
plt.figure(figsize=(14, 8))
sns.boxplot(x='Location', y='Rent', data=filtered_data)
plt.title('Top 20 Expensive Locations by Median Rent Prices')
plt.xlabel('Location')
plt.ylabel('Rent (AED)')
plt.yscale('log')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Identify the most expensive and least expensive areas for renting
# Calculate median rent for each location and sort them
location_rent_stats = df.groupby('Location')['Rent'].median().sort_values()

# Display the least expensive areas
print("Least Expensive Areas for Renting (Top 10):")
print(location_rent_stats.head(10))

# Display the most expensive areas
print("\nMost Expensive Areas for Renting (Top 10):")
print(location_rent_stats.tail(10))

# Least expensive areas
plt.figure(figsize=(12, 6))
least_expensive = location_rent_stats.head(10)
plt.barh(least_expensive.index, least_expensive.values, color='green')
plt.xlabel('Median Rent (AED)')
plt.title('Top 10 Least Expensive Areas for Renting')
plt.show()

# Most expensive areas
plt.figure(figsize=(12, 6))
most_expensive = location_rent_stats.tail(10)
plt.barh(most_expensive.index, most_expensive.values, color='red')
plt.xlabel('Median Rent (AED)')
plt.title('Top 10 Most Expensive Areas for Renting')
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
        lon = df['Longitude'],
        lat = df['Latitude'],
        mode = 'markers',
        marker=dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues'
        )
        ))

fig.update_layout(
        title = 'Geo Spatial View',
        geo=dict(
        scope='asia',  # Set the geographic scope
        showland=True,
        landcolor = "rgb(250, 250, 250)",
        subunitcolor = "rgb(217, 217, 217)",
        countrywidth = 0.5,
        subunitwidth = 0.5,
        showcountries=True,
        countrycolor='darkgray',
        showocean=True,
        oceancolor='lightblue',
        showlakes=True,
        lakecolor='lightblue',
        showrivers=True,
        rivercolor='lightblue',
        showcoastlines=True,
        coastlinecolor='darkblue',
        projection_type='natural earth',
        center=dict(lon=55.2708, lat=25.2048)
    )
)

fig.show()

In [ ]:
# Analyzing the correlation between numerical features and rent prices
numerical_data = df.select_dtypes(include=[np.number])
correlation_matrix = numerical_data.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

# processing data

#### delete null values

In [ ]:
df.dropna(inplace=True)
print(df.isna().sum())

#### Encoding

In [ ]:
obj_columns = df.select_dtypes(include=["object"]).columns
for col in obj_columns:
    uniques = df[col].unique()
    print(col, uniques)

In [ ]:
columns_to_label_encoding = ["Rent_category", "Frequency", "Furnishing", "Purpose"]



columns_to_categorical_encoding = ["Address", "Type", "Location", "City"]

In [ ]:
label_encoder = LabelEncoder()
for col in columns_to_label_encoding:
    df[col] = label_encoder.fit_transform(df[col])

In [ ]:
for col in columns_to_label_encoding:
    print(col, df[col].unique())

In [ ]:
df["Posted_date"] = pd.to_datetime(
    df["Posted_date"], format="%Y-%m-%d", errors="coerce"
).dt.day_of_year

In [ ]:
df["Posted_date"]

***Categorical Ecoding*** *(Target encoder)*

In [ ]:
encoder = ce.TargetEncoder(cols=columns_to_categorical_encoding)
df_encoded = encoder.fit_transform(
    df[columns_to_categorical_encoding], df["Rent_category"]
)

df = pd.concat(
    [df.drop(columns=columns_to_categorical_encoding), df_encoded], axis=1
)

In [ ]:
df[columns_to_categorical_encoding]

## Handling Outilers

#### Handling outliers in 1st column

In [ ]:
print(df.columns[0])

In [ ]:
sns.set_style("darkgrid")
sns.boxplot(data=df["Rent"])

In [ ]:
num_rent_outiers = (df["Rent"] > 1e7).sum()
print(num_rent_outiers)

In [ ]:
rent_mean = df["Rent"].mean()
print(rent_mean)
print(df["Rent"].min(), df["Rent"].max())

In [ ]:
df["Rent"].loc[df["Rent"] > 1e7] = 1e7

In [ ]:
# check outliers again
num_rent_outiers = (df["Rent"] > 1e7).sum()
print(num_rent_outiers)

***Handling outliers in 2nd column***

In [ ]:
print(df.columns[1])

In [ ]:
sns.boxplot(data=df["Beds"])

In [ ]:
print(df["Beds"].min(), df["Beds"].max())

***Handling outliers in 3rd column***

In [ ]:
print(df.columns[2])

In [ ]:
sns.boxplot(data=df["Baths"])

In [ ]:
num_baths_outiers = (df["Baths"] > 10).sum()
print(num_baths_outiers)

In [ ]:
print((df["Baths"]).max())

***Handling outliers in 4th column***

In [ ]:
print(df.columns[3])

In [ ]:
sns.boxplot(data=df["Area_in_sqft"])

In [ ]:
area_mean = (df["Area_in_sqft"]).mean()
print(area_mean)

In [ ]:
num_area_outiers = (df["Area_in_sqft"] > 10000).sum()
print(num_area_outiers)

In [ ]:
(df["Area_in_sqft"]).max()

In [ ]:
df["Area_in_sqft"].loc[df["Area_in_sqft"] > 10000] = 10000

In [ ]:
# check outliers again
num_outiers = (df["Area_in_sqft"] > 10000).sum()
print(num_outiers)

***Handling outliers in 5th column***

In [ ]:
print(df.columns[4])

sns.boxplot(data=df["Rent_per_sqft"])

num_rent_sqft_outliears = (df["Rent_per_sqft"] > 280).sum()
print(num_rent_sqft_outliears)

df["Rent_per_sqft"].loc[df["Rent_per_sqft"] > 280] = 280

# check num outliers again
num_rent_sqft_outliears = (df["Rent_per_sqft"] > 280).sum()
print(num_rent_sqft_outliears)

***Handling outliers in 6th column***

In [ ]:
print(df.columns[5])

sns.boxplot(data=df["Rent_category"])

***Handling outliers in 7th column***

In [ ]:
print(df.columns[6])

sns.boxplot(data=df["Frequency"])

print(
    df["Frequency"].max(),
    df["Frequency"].min(),
)

***Handling outliers in 8th column***

In [ ]:
print(df.columns[7])

sns.boxplot(data=df["Furnishing"])

***Handling outliers in 9th column***

In [ ]:
print(df.columns[8])

sns.boxplot(data=df["Purpose"])

***Handling outliers in 10th column***

In [ ]:
print(df.columns[9])

sns.boxplot(data=df["Posted_date"])

print(df["Posted_date"].min(), df["Posted_date"].max())
num_date_outlin = (df["Posted_date"] > 300).sum()
print(num_date_outlin)

date_mean = df["Posted_date"].mean()
print(date_mean)

df["Posted_date"].loc[df["Posted_date"] > 300] = date_mean

print((df["Posted_date"] > 300).sum())

***Handling outliers in 11th column***

In [ ]:
print(df.columns[10])

sns.boxplot(data=df["Age_of_listing_in_days"])

print(df["Age_of_listing_in_days"].min(), df["Age_of_listing_in_days"].max())
num_age_outlier = (df["Age_of_listing_in_days"] > 200).sum()
print(num_age_outlier)

age_mean = df["Age_of_listing_in_days"].mean()
print(age_mean)

df["Age_of_listing_in_days"].loc[df["Age_of_listing_in_days"] > 200] = 200

print((df["Age_of_listing_in_days"] > 200).sum())

***Handling outliers in 12th column***

In [ ]:
print(df.columns[11])

sns.boxplot(data=df["Latitude"])

print(df["Latitude"].min(), df["Latitude"].max())

***Handling outliers in 13th column***

In [ ]:
print(df.columns[12])

sns.boxplot(data=df["Longitude"])

print(df["Longitude"].min(), df["Longitude"].max())

***Handling outliers in 14th column***

In [ ]:
print(df.columns[13])

sns.boxplot(data=df["Address"])

address_mean = df["Address"].mean()
print(address_mean)

num_outiers = (df["Address"] > 2.5).sum() + (df["Address"] < 1.7).sum()
print(num_outiers)

df["Address"].loc[df["Address"] > 2.5] = address_mean
df["Address"].loc[df["Address"] < 1.7] = address_mean

# check outliers again
num_outiers = (df["Address"] > 2.5).sum() + (df["Address"] < 1.7).sum()
print(num_outiers)

***Handling outliers in 15th column***

In [ ]:
print(df.columns[14])

sns.boxplot(data=df["Type"])

print(df["Type"].min(), df["Type"].max())

***Handling outliers in 16th column***

In [ ]:
print(df.columns[15])

sns.boxplot(data=df["Location"])

print(df["Location"].min(), df["Location"].max())

***Handling outliers in 17th column***

In [ ]:
print(df.columns[16])

sns.boxplot(data=df["City"])

print(df["City"].min(), df["City"].max())

### ***Scaling***

In [ ]:
print(df.columns)

In [ ]:
colums_to_scaling = [
    "Beds",
    "Baths",
    "Area_in_sqft",
    "Rent_per_sqft",
    "Furnishing",
    "Posted_date",
    "Age_of_listing_in_days",
    "Latitude",
    "Longitude",
    "Address",
]

In [ ]:
scaler = MinMaxScaler(feature_range=(1, 2))
scaler.fit(df[colums_to_scaling])

In [ ]:
scaler.fit_transform(df[colums_to_scaling])

In [ ]:
df[colums_to_scaling] = scaler.transform(df[colums_to_scaling])

In [ ]:
df[colums_to_scaling]

In [ ]:
df

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# assume X is your feature matrix and y is your target variable
selector = SelectKBest(f_classif, k=3)  # select top 5 features
X_selected = selector.fit_transform(df[['Beds','Baths','Area_in_sqft','Rent_per_sqft']], df['Rent'])

f = ['Beds','Baths','Area_in_sqft','Rent_per_sqft']
# get the mask of selected features
mask = selector.get_support()

# assume feature_labels is a list of feature labels
selected_feature_labels = [f[i] for i in range(len(mask)) if mask[i]]

print(selected_feature_labels)  # print the labels of the selected features